In [ ]:
#Theodore McCullough, Ishank Aggarwal, Alex Rabanes
#CSC 859. Fall 2022
#Team Project
#Team #1
import sklearn
import numpy as np
import csv
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import joblib
import matplotlib.pyplot as plt
import itertools
from itertools import cycle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay
from sklearn.inspection import permutation_importance
from sklearn.metrics import roc_auc_score
import pandas as pd
import sys


#Input: None
#Processing: Loads wifi data set, extracts‐ Features, and labels
#Output: Label train and test sets, Feature train and test set
def Process_DataSet():
    dataset = pd.read_csv('WiFi Localization Data.csv')
    X = dataset.iloc[:, :-1].values #Creates feature set
    y = dataset.iloc[:, 7].values # Create label set
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=100)
    return [X,X_train, X_test, y_train, y_test]


#Input: None
#Processing: Helper function and called by Create_Forest_Objects function
#Output: Returns a Random Forest Tree Object     
def Create_Forest(forest_size, n_features):
    return RandomForestClassifier(n_estimators = forest_size, criterion='gini',
                                        max_depth=None, min_samples_split=2,min_samples_leaf=1,
                                        min_weight_fraction_leaf=0.0,max_features=n_features, max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0,bootstrap=True,
                                        oob_score=True, n_jobs=None,random_state=None, verbose=0, 
                                        warm_start=False, class_weight=None,ccp_alpha=0.0, max_samples=None)
    
    
#Input: Feature Training Set, Label Training Set, Random Forest Object, Number of trees in forest
#Processing: Plots Random Forest Object, displays, and stores into .png file
#Output: Displayed Random Forest Object, and .png Forest file
def Print_Forest(x_train, y_train, clf, forest_size):
    fn=x_train
    cn=y_train
    fig, axes = plt.subplots(nrows = 1,ncols = forest_size,figsize = (10,2),
dpi=1000)
    if forest_size == 1:
        tree.plot_tree(clf.estimators_[0], max_depth=None, feature_names=["Access Point 1", "Access Point 2", "Access Point 3", "Access Point 4", "Access Point 5", "Access Point 6", "Access Point 7"], class_names=["Room 1", "Room 2", "Room 3","Room 4"], label='all',
                       filled=False, impurity=True, node_ids=False,proportion=False, rounded=False,
                       precision=3, ax=None, fontsize=None);   
        axes.set_title('Tree: ' + str(0), fontsize = 11)
    else:
        for index in range(0, forest_size):
            tree.plot_tree(clf.estimators_[index], max_depth=None,feature_names=["Access Point 1", "Access Point 2", "Access Point 3", "Access Point 4", "Access Point 5", "Access Point 6", "Access Point 7"], 
                           class_names=["Room 1","Room 2", "Room 3", "Room 4"], label='all',filled=False, impurity=True, node_ids=False,proportion=False, rounded=False,precision=3, ax=axes[index], fontsize=None)   
            axes[index].set_title('Tree: ' + str(index), fontsize = 11)
    fig.savefig('rf_trees.png')
    
    
#Input: Random Forest Object (clf), Feature Test Set (x_test), Label Test Set (y_test)
#Processing: Creates labeled Confusion Matrix for Random Forest Object
#Output: PLots Confusion Matrix for Random Forest Object
def Print_Confusion_Matrix(clf, x_test, y_test):
    length = len(x_test)
    y_pred = []
    for index in range(0, length):
        y_pred.append(clf.predict(x_test[[index]]))
    cm = confusion_matrix(y_test, y_pred, labels=clf.classes_, normalize = None)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=["Room 1","Room 2", "Room 3", "Room 4"])
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    
#Input: Trained Random Forest Object (clf), Feature Test Set (x_test), Label Test Set (y_test)
#Processing: Creates an array (y_pred) of predicted labels from x_train, provides y_pred and y_train to F1 function, generates OOB accuracy score for clf
#Output: Prints f1 score and OOB accuracy scores generally for clf
def Print_OOB_F1(x_test, y_test, clf):
    length = len(x_test)
    y_pred = []
    for index in range(0, length):
        y_pred.append(clf.predict(x_test[[index]]))
    f1 = f1_score(y_test, y_pred, labels= [1,2,3,4], average=None)
    f1_add = f1[0] + f1[1]
    f1_average = f1_add/2
    print("F1 score generally:")
    print(f1_average)
    print("OOB score generally:")
    print(clf.oob_score_)
     
        
#Input: Feature Test Set (x_test), Label Test Set (y_test), Trained Random Forest Object (clf)
#Processing: Creates an array (y_pred) of predicted labels from x_train, provides y_pred and y_train to accuracy_score function
#Output: Prints Accuracy score for Trained Random Forest Object
def Print_Accuracy(x_test, y_test, clf):
    length = len(x_test)
    y_pred = []
    for index in range(0, length):
        y_pred.append(clf.predict(x_test[[index]]))
    accuracy = accuracy_score(y_test, y_pred, normalize=True)
    print("Accuracy is:")
    print(accuracy)
    
    
#Input: Feature Test Set (x_test), Label Test Set (y_test), Trained Random Forest Object (clf)
#Processing: Creates an array (y_pred) of predicted labels from x_train, provides y_pred and y_train to classfication_report function
#Output: Prints classification report with F1, Recall, and Accurcy Score
def Print_Report(x_test, y_test, clf):
    length = len(x_test)
    y_pred = []
    for index in range(0, length):
        y_pred.append(clf.predict(x_test[[index]]))
    target_names = ["Room 1", "Room 2", "Room 3", "Room 4"]
    print(classification_report(y_test, y_pred, target_names=target_names,
sample_weight=None, digits = 5))
       
        
#Input: Label Test Set (y_test), Feature Test Set (x_test), Trained Random Forest Object (clf)
#Processing: Uses input to plots true positive rate over false positive rate at different classification thresholds
#Output: Represents plot as a graph (ROC graph)
def Print_ROC(y_test, x_test, clf):
    y_prob = clf.predict_proba(x_test)

    macro_roc_auc_ovo = roc_auc_score(y_test, y_prob, multi_class="ovo", average="macro")
    weighted_roc_auc_ovo = roc_auc_score(
        y_test, y_prob, multi_class="ovo", average="weighted"
    )
    macro_roc_auc_ovr = roc_auc_score(y_test, y_prob, multi_class="ovr", average="macro")
    weighted_roc_auc_ovr = roc_auc_score(
        y_test, y_prob, multi_class="ovr", average="weighted"
    )
    print(
        "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
        "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
    )
    print(
        "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
        "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
    )
    
        
#Input: Trained Random Forest model(clf)
#Processing: Loads feature names, selects top feature types used by clf
#Output: Bar graph of top feature types for clf, and data for top features types    
def Print_MDI(clf):
    feature_names = ["Access Point 1","Access Point 2","Access Point 3","Access Point 4","Access Point 5","Access Point 6","Access Point 7"]
    mdi_importances = pd.Series(clf[-1].feature_importances_, index=feature_names).sort_values(ascending=True)
    ax = mdi_importances.plot.barh()
    ax.set_title("Random Forest Feature Importances (MDI)")
    ax.figure.tight_layout()
   

#Processes data, creates and trains Random Forest
forest_size = int(input("Select number of estimators (trees) (enter 0 to end): "))
n_features = int(input("Select number of features to sample: "))
if forest_size == 0:
    sys.exit("Bye Bye!!")
else:
    clf = Create_Forest(forest_size, n_features)
    X, X_train, x_test, y_train, y_test = Process_DataSet()
    clf.fit(X_train, y_train)
    
    
def Print_Top1Feature():
    n_features = int(input("Select number of features to sample: "))
    dataset = pd.read_csv('WiFi Localization Data.csv')
    labels = ["Access Point 2","Access Point 3","Access Point 4","Access Point 5","Access Point 6","Access Point 7"]
    dataset.drop(columns=labels,index=None)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, 7].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
    classifier = RandomForestClassifier(n_estimators=100, random_state=25, oob_score=True, max_features=n_features)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))
    print(classifier.oob_score_)
    
    
def Print_Top2Features():
    n_features = int(input("Select number of features to sample: "))
    dataset = pd.read_csv('WiFi Localization Data.csv')
    labels = ["Access Point 2","Access Point 3","Access Point 4","Access Point 6","Access Point 7"]
    dataset.drop(columns=labels,index=None)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, 7].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
    classifier = RandomForestClassifier(n_estimators=100, random_state=25, oob_score=True, max_features=n_features)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))
    print(classifier.oob_score_)
    
    
def Print_Top3Features():
    n_features = int(input("Select number of features to sample: "))
    dataset = pd.read_csv('WiFi Localization Data.csv')
    labels = ["Access Point 2","Access Point 3","Access Point 6","Access Point 7"]
    dataset.drop(columns=labels,index=None)
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, 7].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)
    classifier = RandomForestClassifier(n_estimators=100, random_state=25, oob_score=True, max_features=n_features)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))
    print(classifier.oob_score_)
    
    
while(True):
    print()
    print ("Select analysis type: ")
    print ("1. Enter '1' to print Forest: ")
    print ("2. Enter '2' to print Confusion Matrix: ")
    print ("3. Enter '3' to print OOB & F1 score: ")
    print ("4. Enter '4' to print Accuracy: ")
    print ("5. Enter '5' to print Classification Report: ")
    print ("6. Enter '6' to print ROC: ")
    print ("7. Enter '7' to print MDI Graph: ")
    print ("8. Enter '8' to print Classification Report for Top 1 Feature: ")
    print ("9. Enter '9' to print Classification Report for Top 2 Features: ")
    print ("10. Enter '10' to print Classification Report for Top 3 Features: ")
    print ("11. Enter '11' to End Program: ")
    menu_selection = int(input("Select Input Options: "))
    
    if menu_selection == 1:
        Print_Forest(X_train, y_train, clf, forest_size)
        
    elif menu_selection == 2:
        Print_Confusion_Matrix(clf, x_test, y_test)
           
    elif menu_selection == 3:
        Print_OOB_F1(X_train, y_train, clf)
    
    elif menu_selection == 4:
        Print_Accuracy(X_train, y_train, clf)
    
    elif menu_selection == 5:
        Print_Report(X_train, y_train, clf)
        
    elif menu_selection == 6:
        Print_ROC(y_test, x_test, clf)
        
    elif menu_selection == 7:
        Print_MDI(clf)
        
    elif menu_selection == 8:
        Print_Top1Feature()
        
    elif menu_selection == 9:
        Print_Top2Features()
        
    elif menu_selection == 10:
        Print_Top3Features()
        
    elif menu_selection == 11:
        sys.exit("Bye Bye!!")
        
    else:
        print ("Entered wrong selection, try again")
        print (" ")         